In [ ]:
class LearningSurrogateLoss:
    def __init__(self):
        pass
    def predict(self, df, household_idx):
        # Line1
        df = self._prior(df, household_idx)

    def _prior(self, df, household_idx=1):
        if household_idx in [2]:
            preds_time = []
            for _, row in df.iterrows():
                if (row.Time >= 18) & (row.Time <= 40):
                    preds_time.append(0)
                else:
                    preds_time.append(1)
            df["preds_time"] = preds_time
        elif household_idx in [1, 3]:
            preds_time = []
            for _, row in df.iterrows():
                if (row["Time"] >= 24) & (row["Time"] <= 28):
                    preds_time.append(0)
                else:
                    preds_time.append(1)
            df["preds_time"] = preds_time
        return df